<a href="https://colab.research.google.com/github/arpitvaghela/DSA_FL/blob/main/Copy_of_FL_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM time series prediction using Federated Learning

## Imports

In [ ]:
!pip install --quiet --upgrade tensorflow_federated_nightly
!pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

     |████████████████████████████████| 532kB 7.6MB/s 
     |████████████████████████████████| 153kB 10.9MB/s 
     |████████████████████████████████| 1.1MB 18.4MB/s 
     |████████████████████████████████| 3.0MB 43.3MB/s 
     |████████████████████████████████| 174kB 43.1MB/s 
     |████████████████████████████████| 112kB 45.2MB/s 
     |████████████████████████████████| 394.9MB 43kB/s 
     |████████████████████████████████| 10.6MB 40.3MB/s 
     |████████████████████████████████| 471kB 40.6MB/s 
     |████████████████████████████████| 1.3MB 39.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20201106 has requirement absl-py~=0.10, but you'll have absl-py 0.9.0 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20201106 has requirement grpcio~=1.32.0, but you'll have grpcio 1.29.0 which is incompatible.
ERROR: tf-nightly 2.5.0.dev20201106 has requirement numpy~=1.19.2, but you'll have numpy 1

In [ ]:
import collections
import functools
import os
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf

np.random.seed(0)

In [ ]:
import tensorflow_federated as tff

# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Data preprocessing


In [ ]:
#FM = np.fromfile('96_7_20db',dtype=np.float32)
N = 10**6
FM = np.random.randn(N) # normal distributed channel
FM.reshape(-1,1) 

array([[-2.67305145],
       [ 0.24984414],
       [-1.67010768],
       ...,
       [-1.12554283],
       [ 0.95170926],
       [-1.15237806]])

### Bandpower Equation


In [ ]:
def bandpower(signal:np.array)->np.float:
  return np.mean(signal ** 2)

In [ ]:
bandpower(FM)

0.9977254833893389

### awgn Function

Function to add noise to the signal resulting in given S/N ratio

In [ ]:
def awgn(signal:np.ndarray, desired_snr:int):
  """Add AWGN noise to generate signal with given SNR. 
  """
  # Converting the SNR from dB scale to linear scale
  snr_linear = math.pow(10, desired_snr / 10)
  
  # Standard normally distributed noise
  noise = np.random.randn(signal.shape[0], 1)
  
  # Using the boxed formula
  var_signal = bandpower(noise) * snr_linear
  
  # Normalizing the signal to have the given variance
  normalized_signal = math.sqrt(var_signal) * (signal / math.sqrt(bandpower(signal)))
  
  #print("SNR = " + str(10 * math.log10(bandpower(normalized_signal) / bandpower(noise))))
  
  return normalized_signal + noise

## Filtering Data

filtering data points to be in range $10^{-7}< signal< 1$

In [ ]:
FM = FM[np.logical_and(FM > math.pow(10, -7), FM < 1)]
FM = FM.reshape(FM.shape[0], 1)
print("Size of FM: " + str(FM.shape))

Size of FM: (341466, 1)


## Creating Dataset

- take datapoints of size, _samples*sample_size_

- add noise with desire snr

- sample = $[s_1,s_2,\cdots,s_N]$

- Energy detection = $\sum_{i=1}^{N}s_i^2$

- $X[j]$ = $\sum_{i=j*N+1}^{(j+1)*N}s_i^2$

In [ ]:
def create_dataset(signal, desired_snr, samples, sample_size):
  
  # Creating the signal with desired SNR
  snr_signal = awgn(signal[0:samples * sample_size], desired_snr)
  
  # Allocating zeros to the dataset
  X = np.zeros((samples, 1))
  
  for i in range(0, samples):
    
    # Extracting the sample based on sample size
    sampled_signal = snr_signal[i * sample_size : (i + 1) * sample_size]
    
    # Sorting the sampled signal
    sampled_signal = np.sort(sampled_signal, axis=0)
    
    # Energy detection
    E = np.sum(sampled_signal ** 2)
    
    # Assigning values to the dataset
    X[i][0] = E
  
  return X

In [ ]:
%%time
# sample dataset for SNR=4

print(create_dataset(FM[50000:], 4, 15000, 100).shape)

(15000, 1)
CPU times: user 206 ms, sys: 1.88 ms, total: 208 ms
Wall time: 209 ms


In [ ]:
def final_dataset(signal, snr_range, samples_per_snr, sample_size):
  X = {}
  
  for snr in snr_range:
    # Creating dataset for the given SNR
    X_snr = create_dataset(signal, snr, samples_per_snr, sample_size)
    
    # Indexing within the final dataset matrix X
    X[snr] = X_snr
  
  return X

## Generating White Noise Sequence 

In [ ]:
def create_noise_sequence(samples, sample_size):
  
  # Creating white noise sequence of variance 1
  noise = np.random.randn(samples * sample_size, 1)
  
  # Allocating zeros to the dataset
  X = np.zeros((samples, 1))
  
  for i in range(0, samples):
    
    # Extracting the sample based on sample size
    sampled_signal = noise[i * sample_size : (i + 1) * sample_size]
    
    # Sorting the sampled signal
    sampled_signal = np.sort(sampled_signal, axis=0)
    
    # Energy detection
    E = np.sum(sampled_signal ** 2)
    
    # Assigning values to the dataset
    X[i][0] = E
  
  return X

## DataSet LookBack for RNN

In [ ]:
# Function for Chaning the dataset for look back  #linear dataset initially
def create_look_back(X, look_back=1):
  
  # Look back dataset is initialized to be empty
  look_back_X = []
  
  for i in range(len(X) - look_back + 1):
    # Extracting an example from the dataset
    a = X[i:(i + look_back), :]
    
    a = a.flatten() # (For flattening) #1D list
    
    # Appending to the dataset
    look_back_X.append(a)
  
#  look_back_Y = []
    
  # Returning in numpy's array format
  return np.array(look_back_X)

In [ ]:
def dataset_look_back(X_tech, snr_range, look_back):
  X_tech_lb = {}
  
  # Look backs for all SNRs
  for snr in snr_range:
    X_tech_lb[snr] = create_look_back(X_tech[snr], look_back)
  
  return X_tech_lb

In [ ]:
def generate_Dataset(FM,snr_ratio=(0.8, 0.2),sample_size=100,total_sample=100000,lsnr_range=(-20,-4),hsnr_range=(-4,6),look_back = 2,eval=False):

  no_of_sample_hsnr = int((total_sample*snr_ratio[0])//((hsnr_range[1] - hsnr_range[0])/2))
  no_of_sample_lsnr = int((total_sample*snr_ratio[1])//((lsnr_range[1] - lsnr_range[0])/2))
  
  X_FM = {**final_dataset(FM[100000:], range(lsnr_range[0],lsnr_range[1], 2),no_of_sample_lsnr, sample_size),
          **final_dataset(FM[100000:], range(hsnr_range[0],hsnr_range[1], 2), no_of_sample_hsnr, sample_size)}

  X_noise = create_noise_sequence(100000, 100)

  # lookback

  X_FM_lb = dataset_look_back(X_FM, range(-20, 6, 2), look_back)
  X_noise_lb = create_look_back(X_noise, look_back)

  # final X_train and y

  X = X_FM_lb[-20]
  y = []

  for snr in range(-18, 6, 2):
    X = np.concatenate((X, X_FM_lb[snr]), axis=0)

  y = np.ones((X.shape[0], 1))
 
  # print(X.shape)
  # print(X_noise_lb.shape)
  X = np.concatenate((X, X_noise_lb), axis=0)
  y_train = np.concatenate((y, np.zeros((X_noise_lb.shape[0], 1))))

  # reshape
  X_train = np.reshape(X, (-1, 2, 1))
  Y_train = np.reshape(y_train,(-1,1,1))
  return X_train,Y_train

 

In [ ]:
# created dataset
X,Y = generate_Dataset(FM)

In [ ]:
print("Shape of X,Y :",X.shape,Y.shape) # similar to the shape we have

# train,test split
train_size = int(len(X)*0.80)
test_size = len(X) - train_size

X_train, X_test = X[:train_size,:,:],X[train_size:,:,:]
Y_train, Y_test = Y[:train_size,:,:],Y[train_size:,:,:]


print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)


Shape of X,Y : (199986, 2, 1) (199986, 1, 1)
(159988, 2, 1) (39998, 2, 1)
(159988, 1, 1) (39998, 1, 1)


In [ ]:
X_train[0],Y_train[0]

(array([[125.50295881],
        [ 90.16296745]]), array([[1.]]))

## LSTM model and training

In [ ]:
model = tf.keras.models.Sequential([
      tf.keras.layers.LSTM(4,input_dim=1),
      tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4)                 96        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,Y_train,validation_split=0.25,epochs=10,verbose=1)

model.evaluate(X_test,Y_test,verbose=1)

Epoch 1/10
3750/3750 [==============================] - 11s 2ms/step - loss: 0.1304 - accuracy: 0.8213 - val_loss: 0.2549 - val_accuracy: 0.4950
Epoch 2/10
3750/3750 [==============================] - 9s 2ms/step - loss: 0.0799 - accuracy: 0.8705 - val_loss: 0.2163 - val_accuracy: 0.6687
Epoch 3/10
3750/3750 [==============================] - 9s 2ms/step - loss: 0.0761 - accuracy: 0.8748 - val_loss: 0.1795 - val_accuracy: 0.7747
Epoch 4/10
3750/3750 [==============================] - 9s 2ms/step - loss: 0.0757 - accuracy: 0.8757 - val_loss: 0.2273 - val_accuracy: 0.6356
Epoch 5/10
3750/3750 [==============================] - 9s 2ms/step - loss: 0.0748 - accuracy: 0.8769 - val_loss: 0.1776 - val_accuracy: 0.7856
Epoch 6/10
3750/3750 [==============================] - 9s 2ms/step - loss: 0.0752 - accuracy: 0.8757 - val_loss: 0.2497 - val_accuracy: 0.5786
Epoch 7/10
3750/3750 [==============================] - 9s 2ms/step - loss: 0.0756 - accuracy: 0.8751 - val_loss: 0.1531 - val_accuracy

[0.22932171821594238, 0.6486324071884155]

In [ ]:
model.predict([
                [[0.88069264],[0.60346049]]
               ])

array([[0.9999708]], dtype=float32)

## Federated Training and preprocessing centrailized data

### Generating a decentralized data

In [ ]:
T_CLIENTS = 500
CLIENT_SAMPLE_SIZE = 2000
step = len(X)/T_CLIENTS

data_fed = [ tf.data.Dataset.from_tensor_slices(
     
         {"value":X[int(i*step):int((i+1)*step)],"label":Y[int(i*step):int((i+1)*step)]}
      
     
     ) for i in range(T_CLIENTS)]

# client dataset can be accesed as data_fed[ CLIENT_ID ]
example_dataset = data_fed[0]

# def preprocess(data_fed):
#   def batch_format_fn(ele):

example_element = next(iter(example_dataset))

print(example_element["value"].numpy()) # example_element[0] refers to X val
print(example_element["label"].numpy()) # example_element[1] refers to X val


[[125.50295881]
 [ 90.16296745]]
[[1.]]


In [ ]:
# preprocess

NUM_CLIENTS = 200
NUM_EPOCHS = 5
BATCH_SIZE = 1000
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):
  
  def batch_format_fn(element):
    return collections.OrderedDict(
        x=element["value"],
        y=element["label"]
    )
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


In [ ]:

preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch["x"].shape

(1000, 2, 1)

In [ ]:
def make_federated_data(client_data,client_ids):
  return [
      preprocess(client_data[x])
      for x in client_ids
  ]


In [ ]:
sample_clients = [x for x in range(NUM_CLIENTS)]

federated_train_data = make_federated_data(data_fed, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))


Number of client datasets: 200
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 2, 1)), (y, (None, 1, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.float64)])>


### Creating the Iterative process

In [ ]:
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

def model_fn():
  keras_model = tf.keras.models.Sequential([
      tf.keras.layers.LSTM(4,input_dim=1),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return tff.learning.from_keras_model(
    keras_model,
    input_spec=preprocessed_example_dataset.element_spec,
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.BinaryCrossentropy()]
  )

iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

str(iterative_process.initialize.type_signature)


'( -> <model=<trainable=<float32[1,16],float32[4,16],float32[16],float32[4,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = iterative_process.initialize()

def keras_evaluate(state, round_num):
  state.model.assign_weights_to(model)
  loss, accuracy = model.evaluate(example_dataset, steps=2, verbose=0)
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))

### Iterations

In [ ]:
NUM_ROUNDS = 5
for round_num in range(1, NUM_ROUNDS):
  # 200 clients of 500 clients are selected for training in various rounds
  # there are 20 rounds and each round takes data from 10 clients
  clients = [round_num*x for x in range(1,11)]
  federated_train_data = make_federated_data(data_fed, clients)
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

keras_evaluate(state, NUM_ROUNDS + 1)

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_crossentropy', 0.5536838), ('loss', 0.18079454)]))])
round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_crossentropy', 0.55420613), ('loss', 0.18109746)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_crossentropy', 0.5518121), ('loss', 0.17995524)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_crossentropy', 0.54022455), ('loss', 0.1742653)]))])


ValueError: ignored